In [18]:
# HPC deployment
SPARK_MASTER = 'spark://cm029:45471'
FILE_DIR = '/scratch/si2073/output'
TARGET_PATH = '/scratch/tmv7269/datasets'
EXAMPLE_FILE = FILE_DIR + '/enwiki-20230401-pages-meta-history10.xml-p4045403p4096288.csv'

In [4]:
import pandas as pd
import numpy as np
import re
import datasets
import os
from datasets import load_dataset
from pathlib import Path
from huggingface_hub import snapshot_download

os.environ["HF_HOME"] = TARGET_PATH
os.environ["HF_DATASETS_CACHE"] = TARGET_PATH

In [6]:
snapshot_download(repo_id="Cohere/wikipedia-22-12-en-embeddings", 
                  local_dir=TARGET_PATH,
                  cache_dir=TARGET_PATH,
                  repo_type="dataset")

Fetching 256 files:   0%|          | 0/256 [00:00<?, ?it/s]

'/scratch/tmv7269/datasets'

In [13]:
# setting up spark
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
import pyspark

spark = SparkSession.builder.master(SPARK_MASTER) \
                            .config('spark.executor.memory', '4G') \
                            .config('spark.driver.memory', '60G') \
                            .config("spark.memory.offHeap.enabled", True) \
                            .config("spark.memory.offHeap.size","16g") \
                            .appName('ProcessData') \
                            .getOrCreate()
spark.sparkContext.setLogLevel("Off")
spark

In [27]:
# Load Cohere's full dataset
drop_columns = ['text', 'id', 'url', 'title', 'langs', 'paragraph_id', 'views']
id_emb_map = spark.read.parquet(f"{TARGET_PATH}/data").drop(*drop_columns).drop_duplicates(subset=['wiki_id'])
id_emb_map.show()

+-------+--------------------+
|wiki_id|                 emb|
+-------+--------------------+
|   1088|[0.24366963, -0.6...|
|   1580|[0.0684566, 0.260...|
|   1645|[0.3224766, 0.458...|
|   2122|[0.111770935, 0.4...|
|   2866|[-0.2080042, -0.2...|
|   3175|[0.16152096, 0.30...|
|   3794|[0.051881086, 0.4...|
|   3997|[0.27129447, 0.53...|
|   4101|[0.34501484, -0.1...|
|   4519|[0.33576313, 0.24...|
|   5300|[0.3257199, -0.34...|
|   6336|[0.38157865, 0.15...|
|   6357|[0.39418507, -0.2...|
|   6466|[-0.1551857, 1.00...|
|   6620|[0.12537971, 0.51...|
|   6654|[-0.105314195, 0....|
|   7253|[0.25702828, 0.55...|
|   7554|[0.2620884, 0.488...|
|   7833|[0.24061485, 0.41...|
|   8086|[0.3114842, -0.26...|
+-------+--------------------+
only showing top 20 rows



In [36]:
id_emb_map.count()

5745033

# Reading in editors csv

In [19]:
def lst_to_str(x):
    return re.sub(r'[^ \w+]', '', x).split()

columns = ['title', 'text', 'username', 'article_id']
df = pd.read_csv(EXAMPLE_FILE, names=columns, skiprows=1).drop(columns='text')
df['username'] = df.username.apply(lst_to_str)

In [20]:
len(df)

30246

In [21]:
spark_df = spark.createDataFrame(df)

In [22]:
spark_df.head()

Row(title='Foster Air Force Base', username=['Dirtydan667', 'Dirtydan667', 'RussBot', 'Ktr101', 'Ktr101', 'Ktr101', 'Ktr101', 'Ndunruh', 'PigFlu', 'Oink', 'Bwmoll3', 'LilHelpa', 'ColoBill4', 'ColoBill4', 'Ground', 'Zero', 'Bwmoll3', 'Bwmoll3', 'Bwmoll3', 'Bwmoll3', 'Bwmoll3', 'Bwmoll3', 'Bwmoll3', 'Bwmoll3', 'Bwmoll3', 'Bwmoll3', 'Bwmoll3', 'Bwmoll3', 'Bwmoll3', 'Bwmoll3', 'Bwmoll3', 'Bwmoll3', 'Bwmoll3', 'Bwmoll3', 'Bwmoll3', 'Bwmoll3', 'Bwmoll3', 'Bwmoll3', 'Bwmoll3', 'Bwmoll3', 'Bwmoll3', 'Bwmoll3', 'Bwmoll3', 'Bwmoll3', 'Bwmoll3', 'Bwmoll3', 'Bwmoll3', 'Bwmoll3', 'Bwmoll3', 'Bwmoll3', 'Bwmoll3', 'Bwmoll3', 'Bwmoll3', 'Vanished', 'user', '31415926535897932384626433', 'Firsfron', 'Bwmoll3', 'Bwmoll3', 'Bwmoll3', 'Bwmoll3', 'Bwmoll3', 'Bwmoll3', 'DiiCinta', 'Bwmoll3', 'Bwmoll3', 'Backspace', 'Rich', 'Farmbrough', 'Mark', 'Sublette', 'Mark', 'Sublette', 'Mark', 'Sublette', 'Kumioko', 'renamed', 'Ulric1313', 'Mark', 'Sublette', 'Ulric1313', 'Mark', 'Sublette', 'Lightmouse', 'Gcstackmone

In [40]:
from pyspark.sql.functions import explode, collect_list, count
from collections import Counter

exploded = spark_df.select(spark_df.article_id, explode(spark_df.username).alias('username'))
collapsed = exploded.groupBy("username", "article_id").agg(count("article_id").alias('count'))
collapsed = collapsed.join(id_emb_map,collapsed.article_id ==  id_emb_map.wiki_id,"inner")
collapsed = collapsed.drop('article_id', 'wiki_id')
collapsed.show()

+---------------+-----+--------------------+
|       username|count|                 emb|
+---------------+-----+--------------------+
|TheRedPenOfDoom|    2|[-0.159246, 0.024...|
|      Waacstats|    1|[-0.159246, 0.024...|
|           Saji|    1|[-0.159246, 0.024...|
|      KasparBot|    1|[-0.159246, 0.024...|
|       Kathanar|    2|[-0.159246, 0.024...|
|    Shyamsunder|    3|[-0.159246, 0.024...|
|        Hockett|    1|[-0.159246, 0.024...|
|       PrimeBOT|    1|[-0.159246, 0.024...|
|Pancakeslover11|    1|[-0.159246, 0.024...|
|             NG|    1|[-0.159246, 0.024...|
|         Srivin|    1|[-0.159246, 0.024...|
|        Pkbwcgs|    1|[-0.159246, 0.024...|
|          Tachs|    3|[-0.159246, 0.024...|
|     KolbertBot|    1|[-0.159246, 0.024...|
|      LivingBot|    1|[-0.159246, 0.024...|
|        Neils51|    2|[-0.159246, 0.024...|
|          Great|    1|[-0.159246, 0.024...|
|        Jim1138|    1|[-0.159246, 0.024...|
|        Cydebot|    4|[-0.159246, 0.024...|
|      Olf

In [41]:
editors_collapsed = collapsed.groupby('username').agg(collect_list('emb').alias('embs'), collect_list('count').alias('counts'))
editors_collapsed.show()

+-------------+--------------------+--------------------+
|     username|                embs|              counts|
+-------------+--------------------+--------------------+
|           07|[[0.28359607, 0.2...|[1, 1, 2, 1, 2, 3...|
| 11deadpool11|[[0.18078326, 0.2...|                 [1]|
|      4meter4|[[-0.027774015, 0...|[1, 2, 1, 1, 1, 2...|
|      4owowoo|[[0.6785187, -0.1...|                 [1]|
|    A1octopus|[[0.08518514, 0.2...|                 [2]|
|          Aa4|[[0.04368933, -0....|                 [2]|
|AaronzAccount|[[0.049395856, 0....|                 [3]|
|          Ahc|[[0.19575398, 0.1...|                 [1]|
|      Ahendra|[[-0.026980087, 0...|[4, 119, 188, 1, ...|
|Ajaydhivyanth|[[0.07035242, -0....|                 [1]|
|         Ales|[[-0.037420806, 0...|                 [2]|
|   Algebraist|[[0.3974199, 0.05...|           [1, 1, 1]|
|     Alienpmk|[[0.30616087, -0....|                 [3]|
|      Amazing|[[0.20678379, -0....|              [1, 1]|
|Amershakigaga

In [42]:
from typing import List, Tuple
import numpy as np
import math
import itertools

# each editor has a list of ([emb], count) of the things they edited
# assuming all id in list is unique
def calculate_editor_variance(matrix : List[List[int]], weights : List[int]):
    return calculate_multid_distance(matrix, weights)

"""
Take in matrix of article x embedding and article's respective weight, return their variance in k-dimension
this is easier to interpret when using LDA (since you can see how far editors stray on certain topic, 
    assuming topic itself is indepedent)
With vaguer embedding, it's better to use the L2 norm and compress it to a single number
"""
def calculate_multid_variance(matrix, weights):
    average = np.average(matrix, weights=weights, axis=0)
    variance = np.average((matrix-average)**2, weights=weights, axis=0)
    return np.sqrt(variance)

"""
Calculate L2 distance for each d-dimension point
"""
def calculate_multid_distance(matrix, weights):
    average = np.average(matrix, weights=weights, axis=0)
    l2 = np.linalg.norm(matrix-average, axis=1)
    return np.average(l2)

In [47]:
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

calc_variance_udf = udf(calculate_editor_variance, IntegerType())
editors_variance = editors_collapsed.withColumn('variance', calc_variance_udf('embs', 'counts'))
editors_variance.show()

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/ext3/spark-3.1.2-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/worker.py", line 604, in main
    process()
  File "/ext3/spark-3.1.2-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/worker.py", line 596, in process
    serializer.dump_stream(out_iter, outfile)
  File "/ext3/spark-3.1.2-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/serializers.py", line 211, in dump_stream
    self.serializer.dump_stream(self._batched(iterator), stream)
  File "/ext3/spark-3.1.2-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/serializers.py", line 132, in dump_stream
    for obj in iterator:
  File "/ext3/spark-3.1.2-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/serializers.py", line 200, in _batched
    for item in iterator:
  File "/ext3/spark-3.1.2-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/worker.py", line 450, in mapper
    result = tuple(f(*[a[o] for o in arg_of